# Another method of doing simulations

We'll quickly look at a different method of doing the simulations you saw last session. This method is a bit trickier to think about than using strings like `CORRECT` or `INCORRECT`, but you might see it used in other peoples' code, so it is important to understand.

Remember the scenario from the last session:

>You have a friend who claims that through [extrasensory perception]>(https://en.wikipedia.org/wiki/Extrasensory_perception#:~:text=Extrasensory%20perception%20(ESP)%2C%20also,but%20sensed%20with%20the%20mind.) they have the power to predict which card will be drawn next from a deck of cards.

>You'd like to test this claim, in an unbiased fashion.

>You draw, one by one, twenty cards from a deck. On each draw, your friend makes a guess as to the card's identity (e.g. Ace of Spades; 3 of Clubs etc.) before seeing the card. You then show them the true identity of the card, and then put the card back in the deck before the next draw. You both record the number of cards they correctly guessed.

>Your friend guesses 8 of the 20 cards correctly.

>Is that result surprising? Is your friend guessing randomly, or is there evidence they have some extrasensory ability e.g. the ability to "perceive" what the card is, in a non-random way?

>If you friend does *not* have such an ability, then the chance of correctly guessing on each draw is $\frac{1}{52}$, as there are 52 cards in a standard deck.

Remember that our simulation model is that your friend's guesses are random? (E.g. we are in the Null World, where your friend does not have extrasensory abilities).

That is, for any guess your friend has a $\frac{1}{52}$ probability of being correct.

Here is another way of doing the simulation - a bit more complex than the ones we saw last time, but important to understand. It involves simulating the guesses by taking 20 random numbers between 0 and 1. We then check if each number is lower than $\frac{1}{52}$, and we take a `True` to mean a correct guess.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# set up the random number generator
rng = np.random.default_rng()

In [ ]:
# record how many guesses our friend made
number_of_guesses = 20

number_of_guesses

In [ ]:
# record the proportion of guesses our friend got correct
proportion_of_correct_guesses = 8/number_of_guesses
proportion_of_correct_guesses

In [ ]:
# simulate 20 random guesses
random_guesses = rng.uniform(0, 1, size=number_of_guesses)
random_guesses

In [ ]:
# count the number of simulated correct guesses
simulated_correct_guesses = np.count_nonzero(random_guesses < 1/52)
simulated_correct_guesses

In [ ]:
# we can also use the following to count the number of correct guesses
sum(random_guesses < 1/52)

We stored your friend's correct guesses as a proportion e.g. $\frac{1}{52}$. We can get the proportion of correct simulated guesses this way:

In [ ]:
# we can get the proportion/percentage of correct guesses by doing the following
prop_simulated_correct_guesses = simulated_correct_guesses/number_of_guesses
prop_simulated_correct_guesses

Now, let's practice using this method to perform a simulation with 10_000 trials. 

In [ ]:
# perform your simulation here

This rest of this notebook will re-vist your **permutation testing** skills (as well as your `pandas` skills). We use permutation testing (as opposed to simulation) when our observed statistic is a *difference between two groups*. Remember, we use simulation when testing a single sample, as we saw in the last session.

Just FYI - there are other contexts we can use permutation testing, but the *between groups* context is the primary one you've seen. 

# Does a treatment work?

Let's consider the following situation:

- there is a disease with unpleasant symptoms
- a pharmaceutical company rep tells you they have a drug treatment that can reduce the symptom severity (e.g. make people feel better)
- they have conducted a study involving participants who have the disease. Participants were randomly assigned to the placebo group (where they received a fake treatment) and to the treatment group (where they received the drug)
- the pharmaceutical company rep says there that the drug group has lower symptom severity, at the end of the study

Let's have a look at the data and see if we should believe the pharmaceutical rep taht the drug is effective at reducing symptom severity...

**Note**: the data is real experimental data, but the meaning has been changed (for increased clarity; the actual data is from [this dataset](https://github.com/lisds/textbook/blob/main/data/mosquito_beer.csv)).

Here is the meaning of each column:

- `participant_ID`: a unique identifier for each participant
- `group`: whether the participant was in the `placebo` or `drug` group
- `symptom_severity`: the symptom severity score of the the participant, at the end of the study

In [ ]:
# import the dataset
df = pd.read_csv("data/drug_placebo.csv")
         
df         

Before we start to look at the data, let's clean it. You'll notice there is a "junk" first column. Let's remove that in the cell below:

In [ ]:
# clean the data by removing the first column

The first thing we want to check, is whether the average symptom severity between the two groups is different, as the pharmaceutical company rep says.

The code cells below describe the process for doing this:

In [ ]:
# store the data for the placebo group, in a variable called `placebo_group`

In [ ]:
# store the data for the drug group, in a variable called `drug_group`

In [ ]:
# calculate the mean of the placebo group

In [ ]:
# calculate the mean of the drug group

In [ ]:
# calculate the difference between the means (let's discuss which direction of subtraction makes the most sense,
# either is fine, but one may be easier to interpret)

Let's plot the data. We're going to want to plot each group separately (e.g. to see if there is a difference between the two groups).

You may also consider using `plt.axvline()` to plot the group means.

In [ ]:
# plot the data here

Are you convinced by the difference in means - is the treatment effective? Let's discuss.

### The Logic of Permutation Testing

Let's imagine a skeptic arguing with the pharmaceutical company rep. The skeptic refuses to accept that the difference in means indicates that the treatment works.

The skeptic's argument is that the difference we see is just a quirk in this sample - the difference is not big enough to convince us that the drug would work if we gave it to other people with the disease outside of this sample. E.g. if we gave it to others from the *population* of people who suffer with the disease.

Let's think about what the skeptic is saying here, they are saying that the treatment makes **no difference**. One way of thinking of what the skeptic is saying is this:

- we have only one sample from the population of all people with the disease
- if we had gotten a different sample, we would have observed no difference in the syptom severity scores, because the drug is ineffective
- it merely happens that through random chance we got a sample where we observe a difference **but we would not observe thsi difference consistently if we repeated the study with new samples from the same population**

The logic of permutation testing is built on this skeptical claim. If the treatment makes no difference, then the group identification labels `placebo` and `drug` should be **exchangable** between the groups.

By shuffling the group labels, we can simulate *drawing a new random sample under the assumption that the treatment is ineffective*. 

The skeptic is saying that The Null World - where the treatment does not work - is a good model of the real world. Permutation testing let's us simulate the Null World by shuffling the group labels, thus removing any association between the treatment and the symptom severity scores.

If we shuffle a large number of times - and recalculate the difference in average symptom severity score each time - we can then build a distribution of the sorts of differences we would expect if the treatment is ineffective. 

Let's do this in the cell below:

In [ ]:
# perform the permutation test here

In [ ]:
# plot the results of the permutation test here

In [ ]:
# calculate your p-value here

# Political views 

The last permutation test we did involved testing for a difference between groups using a numerical score (`symptom_severity`).

However, you will encounter other types of data, for which you can use a permutation test.

Let's look at the following data on abortion views and political party affiliation (again, this is based on real data, but the meaning has been changed e.g. the original data was not about abortion views).

Let's assume the data is a random sample from a small town.

We want to know if being `republican` or being `democrat` have the same (or different) association with being `pro_abortion`.

The Null World would be a world where *there is no difference in the proportion of republicans who are pro-abortion relative to the proportion of democrats who are pro abortion*:

In [ ]:
# read in the data
df2 = pd.read_csv('data/abortion_views_rep_dem.csv')

df2

In [ ]:
# make the dataframe easier to index
df2.set_index("pro_abortion", inplace=True)
df2

In [ ]:
# get the number of republcian Yes's in its own variable
n_yes_republican = df2.loc['Yes', 'republican']
n_yes_republican

In [ ]:
# get the number of republcian No's in its own variable
n_no_republican = df2.loc['No', 'republican']
n_no_republican

In [ ]:
# get the total number of republicans
n_republican = df2.loc['out of', 'republican']
n_republican

In [ ]:
# get the number of democrat Yes's
n_yes_democrat = df2.loc['Yes', 'democrat']
n_yes_democrat

In [ ]:
# get the number of democrat No's
n_no_democrat = df2.loc['No', 'democrat']
n_no_democrat

In [ ]:
# get the total number of democrts
n_democrat = df2.loc['out of', 'democrat']
n_democrat

We can now calculate the difference in proprtions:

In [ ]:
n_yes_democrat/n_democrat - n_yes_republican/n_republican

In [ ]:
df2

In [ ]:
# the same calculation using numbers straight from the table
54/66 - 64/249 

Now, you'll notice there's not much for us to shuffle in this data!

That's because the data is *aggregated* - e.g. it is a count of the number of people with specific characteristics (e.g. being `republican` or `democrat`, or being pro abortion or not.

In order to perform a permutation test here - e.g. to simulate the null world where there is no difference in the proportion of republicans who are pro-abortion relative to the proportion of democrats who are pro-abortion - we need to recreate the raw data.

**The raw data is a dataframe with one row for each person counted in the original dataframe.**

The function `np.repeat()` is very useful here, for recreating the raw data:

In [ ]:
# a useful function to use here, for recreating the raw data
np.repeat('republican', 20)

In [ ]:
# create the labels for the republicans
republican_labels = np.repeat('republican', n_republican)
yes_republican = np.repeat('Yes', n_yes_republican)
no_republican = np.repeat('No', n_no_republican)

In [ ]:
# stick the republican yes's and no's together
republican_yes_no = np.append(yes_republican, no_republican)

In [ ]:
# create a dataframe for the republican data
republican_df = pd.DataFrame()
republican_df['political_party'] = republican_labels
republican_df['pro_abortion'] = republican_yes_no

republican_df

In [ ]:
# create the labels for the democrats
democrat_labels = np.repeat('democrat', n_democrat)
yes_democrat = np.repeat('Yes', n_yes_democrat)
no_democrat = np.repeat('No', n_no_democrat)
democrat_yes_no = np.append(yes_democrat, no_democrat)

In [ ]:
# create a dataframe for the democrats
democrat_df = pd.DataFrame()
democrat_df['political_party'] = democrat_labels
democrat_df['pro_abortion'] =democrat_yes_no

democrat_df

In [ ]:
# create the raw data dataframe
raw_data = pd.concat([republican_df, democrat_df])

raw_data

In [ ]:
# check that the raw data corresponds to the original data
pd.crosstab(raw_data['pro_abortion'], raw_data['political_party'])

In [ ]:
# show the original data for comparison
df2

We can now shuffle the raw data to generate a simulated table of counts - this simulates taking a new sample from the Null World, where there is no association between party membership and abortion views:

In [ ]:
# get a simulated table 
simulated_counts = pd.crosstab(np.random.permutation(raw_data['pro_abortion']), raw_data['political_party'])
simulated_counts

In [ ]:
# simulated statistic
simulated_counts.loc['Yes', 'democrat']/simulated_counts['democrat'].sum() -  simulated_counts.loc['Yes', 'republican']/simulated_counts['republican'].sum() 

We can now perform a permutation test by shuffling the raw data:

In [ ]:
# do you permutation test here

# More political views

You can practice the same test using the data below, e.g. testing whether being `democrat` or being `independent` have the same (or different) association with being `pro_abortion`:

*Note*: please feel free to send me some screenshots via teams of your code and simulation - I am happy to informally mark your code this way :)

In [ ]:
# read in the democrat/independent data
df3 = pd.read_csv('data/abortion_views_dem_ind.csv')
                  
df3

In [ ]:
# "explode" your data to the raw data here

In [ ]:
# perform your simulation test here